In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

## Data Import

In [ ]:
def load_data(filename):
    diagnostic_df = pd.read_csv(f'{filename}',
                                 dtype={'dtcbase': 'object', 'dtcfull': 'object', 'odomiles': 'float64'}, 
                                 low_memory=False)
    return diagnostic_df

## Feature Engineering: Consultation ID
# !Add description!

In [ ]:
def initiate_diagnostic_consultation(diagnostic_df):
    diagnostic_df['timestamp'] = pd.to_datetime(diagnostic_df['sessiontimestamp'])
    diagnostic_df['date'] = diagnostic_df['timestamp'].dt.date
    
    diagnostic_df.sort_values(['anonymised_vin', 'date'], kind='mergesort', inplace=True)
    diagnostic_df['consultationid'] = (diagnostic_df['anonymised_vin'] != diagnostic_df['anonymised_vin'].shift()).astype(int)
    diagnostic_df['consultationid'] += (diagnostic_df['date'] != diagnostic_df['date'].shift()).astype(int)
    diagnostic_df['consultationid'] = diagnostic_df['consultationid'].cumsum()
    return diagnostic_df

## Data Restructuring: Derive Vehicle Data, Diagnostic Reads and Subsequent Diagnostic Actions
# !Add description!

In [ ]:
def derive_vehicle_state_data(diagnostic_df):
    cols_to_keep = [col for col in diagnostic_df.columns if col not in ['otxsequence', 'date', 'sessionid', 'timestamp']]
    vehicle_current_state_df = diagnostic_df[diagnostic_df['otxsequence'] == 'G2725772'][cols_to_keep].copy()

    cols_to_keep = ['anonymised_vin', 'consultationid', 'timestamp', 'otxsequence']
    diagnostic_actions_df = diagnostic_df[diagnostic_df['otxsequence'] != 'G2725772'][cols_to_keep].copy()

    diagnostic_df = vehicle_current_state_df.merge(diagnostic_actions_df, how='inner', on=['anonymised_vin', 'consultationid'])
    return diagnostic_df

## Merge Warranty Data
# !Add description!

In [ ]:
def merge_diag_warr_data(diagnostic_df, warranty_df):
    diagnostic_df['timestamp'] = pd.to_datetime(diagnostic_df['timestamp'], utc=True)
    warranty_df['i_incident_date'] = pd.to_datetime(warranty_df['i_incident_date'], utc=True)

    warranty_df = warranty_df.rename(columns={'anonymised_vin': 'warranty_anonymised_vin'})

    df_list = []
    for idx, row in diagnostic_df.iterrows():
        vin = row['anonymised_vin']
        diag_time = row['timestamp']

        mask = ((warranty_df['warranty_anonymised_vin'] == vin) &
                (warranty_df['i_incident_date'] >= diag_time) &
                (warranty_df['i_incident_date'] <= diag_time + pd.Timedelta(days=7)))

        temp_warranty_df = warranty_df[mask]

        if temp_warranty_df.empty:  # If no matching warranty claim is found, create a row with placeholders
            merged_row = row.copy()
            merged_row = pd.concat([merged_row, pd.Series([np.nan]*len(warranty_df.columns), index=warranty_df.columns)], axis=0)

        else:
            for _, warranty_row in temp_warranty_df.iterrows():
                merged_row = pd.concat([row, warranty_row])

        df_list.append(merged_row)

    merged_df = pd.concat(df_list, axis=1).T
    return merged_df

## Feature Engineering: Derive Temporal Features

In this section, we perform feature engineering on the 'timestamp' field to extract valuable temporal information about each diagnostic activity and some additional features from the vehicle's 'warrantydate' and 'builddate' data fields. The temporal features we derive are:

1. **Year**: The year the diagnostic activity was performed. This can help detect yearly trends in the data.
2. **Month**: The month the diagnostic activity was performed. This can help identify any monthly patterns.
3. **Day of Week**: The day of the week the diagnostic activity was performed. This can reveal weekly trends, such as certain activities being more common on certain days of the week.
4. **Week of Year**: The ISO week number of the year the diagnostic activity was performed. This can provide a more granular view of yearly trends.
5. **Time Since Last Activity**: The time in seconds since the last diagnostic activity for each consultation. This can help gauge the frequency of activities.
6. **Elapsed Time**: The time in seconds since the first diagnostic activity in each consultation. This can provide insight into the duration of consultations.
7. **Season of the Year**: The season (Winter, Spring, Summer, Autumn) when the diagnostic activity was performed. This can help identify seasonal trends, such as certain activities being more common in certain seasons.
8. **Age of Vehicle at the Time of Diagnostic Session**: The vehicle's age in years at the time of each diagnostic session. This might be a useful feature because older vehicles might have different diagnostic needs than newer ones.
9. **Time in Days since Warranty Started**: This feature might be useful as vehicles might have different diagnostic needs before and after their warranty starts. Also, customers might behave differently before and after their warranty starts.

The resulting dataframe now contains several new features that provide additional temporal context about each diagnostic activity.

In [ ]:
def derive_temporal_data_features(merged_df):
    merged_df.sort_values(['consultationid', 'timestamp'], inplace=True)

    merged_df['year'] = merged_df['timestamp'].dt.year
    merged_df['month'] = merged_df['timestamp'].dt.month
    merged_df['dayOfWeek'] = merged_df['timestamp'].dt.dayofweek
    merged_df['weekOfYear'] = merged_df['timestamp'].dt.isocalendar().week
    merged_df['timeSinceLastActivitySec'] = (merged_df.groupby('consultationid')['timestamp'].diff().dt.total_seconds()).fillna(0)
    merged_df['elapsedTimeSec'] = merged_df.groupby('consultationid')['timestamp'].transform(lambda x: (x - x.min())).dt.total_seconds()

    def month_to_season(month):
        if month in [12, 1, 2]:
            return 'Winter'
        elif month in [3, 4, 5]:
            return 'Spring'
        elif month in [6, 7, 8]:
            return 'Summer'
        else:
            return 'Autumn'
            
    merged_df['season'] = merged_df['month'].apply(month_to_season)

    merged_df['builddate'] = pd.to_datetime(merged_df['builddate'], format='%Y-%m-%d').dt.tz_localize('UTC')
    merged_df['warrantydate'] = pd.to_datetime(merged_df['warrantydate'], format='%Y-%m-%d').dt.tz_localize('UTC')
    merged_df['vehicleAgeAtSession'] = (merged_df['timestamp'] - merged_df['builddate']).dt.days / 365
    merged_df['daysSinceWarrantyStart'] = (merged_df['timestamp'] - merged_df['warrantydate']).dt.days

    merged_df.drop(columns=['builddate', 'warrantydate', 'dtcdescription', 'v_warr_date_event', 'i_p_css_description',
                            'i_original_ccc_description', 'i_cpsc_description', 'i_css_description', 'ic_customer_verbatim',
                            'ic_technical_verbatim', 'i_incident_date', 'ic_accepted_date', 'warranty_anonymised_vin'],
               inplace=True)
    return merged_df

## Feature Engineering: Removing Outlier Diagnostic Activities

In our dataset, certain diagnostic activities performed by the technicians are extremely common and are recorded in virtually every consultation. While these activities are a routine part of the consultation process, they do not carry significant diagnostic information for our model, and therefore, may not be useful in predicting recommendations. For instance, the 'CONSULTATION_START' activity is logged in every consultation but doesn't contribute meaningful information towards diagnosing a specific vehicle issue.

To identify and remove these non-informative activities, we follow a statistical outlier detection approach:

1. **Calculate Commonality**: First, we calculate the commonality score for each activity, which is the frequency of the activity divided by the total number of activities.

2. **Calculate Mean and Standard Deviation**: We then calculate the mean and standard deviation of these commonality scores.

3. **Identify Outliers**: Any activity whose commonality score lies beyond two standard deviations from the mean is considered an outlier. This threshold is based on the empirical rule, which states that for a normal distribution, about 95% of the data lies within two standard deviations of the mean.

4. **Remove Outliers**: Finally, we remove these outlier activities from our dataset, leaving us with a set of activities that are varied enough to provide meaningful information for our model.

In [ ]:
def remove_outlier_diagnostic_activities(merged_df):
    activity_commonality = merged_df.value_counts('otxsequence')/merged_df['otxsequence'].count()
    activity_commonality = activity_commonality.reset_index()
    activity_commonality.columns = ['otxsequence', 'commonalityScore']

    mean = activity_commonality.commonalityScore.mean()
    std = activity_commonality.commonalityScore.std()
    print(f'MEAN: {mean}  STD: {std}')

    lower = mean - (2 * std)
    upper = mean + (2 * std)

    # Identify the outliers by checking for commonality score less than or greater than lower and upper bounds respectively.
    outliers_condition = (activity_commonality.commonalityScore < lower) | (upper < activity_commonality.commonalityScore)
    most_common_activities = activity_commonality[outliers_condition]

    print(f"Most common activities with their commonality score (activities to be removed):\n{most_common_activities}")
    
    # Remove identified outlier (the most common) activities
    num_records_initial = len(merged_df)
    merged_df = merged_df[~merged_df.otxsequence.isin(most_common_activities.otxsequence)]

    print(f'Number of records removed: {num_records_initial - len(merged_df)}')
    return merged_df

## Data Cleaning: Removing Duplicate Records

In this step of the data preprocessing, we aim to remove any duplicate entries in the dataset.

We utilize the `drop_duplicates()` function from pandas library for this purpose. The `inplace=True` parameter ensures that the operation is performed on the dataset directly, without the need to assign the result to a new variable.

In [ ]:
def remove_duplicates(merged_df):
    num_records_initial = len(merged_df)
    merged_df.drop_duplicates()

    print(f'Number of duplicate records removed: {num_records_initial - len(merged_df)}')
    return merged_df

## Data Cleaning: Identify and Handle Missing Values
# !UPDATE!
The isna().sum() call returns a pandas Series with column names as the index and the count of NaN values as the values. We then filter this series to only include columns with more than 0 NaN values and print them.

In [ ]:
def handle_missing_vals(merged_df):
    # Apply 'Unknown' category to all missing values on categorical variables
    unordered_cat_cols = ['anonymised_vin', 'consultationid', 'otxsequence', 'model', 'modelyear', 'driver',
                          'plant', 'engine', 'transmission', 'module', 'dtcbase', 'faulttype', 'dtcfull',
                          'softwarepartnumber', 'hardwarepartnumber', 'i_p_css_code', 'i_original_ccc_code', 
                          'i_original_vfg_code','i_original_function_code', 'i_original_vrt_code', 'i_current_vfg_code',
                          'i_current_function_code', 'i_current_vrt_code',	'i_cpsc_code', 'i_cpsc_vfg_code',
                          'i_css_code', 'v_transmission_code','v_drive_code', 'v_engine_code', 'ic_repair_dealer_id',
                          'ic_eng_part_number', 'ic_serv_part_number','ic_part_suffix', 'ic_part_base', 'ic_part_prefix', 
                          'ic_causal_part_id', 'ic_repair_country_code', 'ic_replaced_yn']

    for col in unordered_cat_cols:
        merged_df[col] = merged_df[col].fillna('Unknown')

        
    # If daysSinceWarrantyStart NaN, then warrantydate was empty, meaning it is likely that warranty 
    # has not started on the vehicle
    merged_df['daysSinceWarrantyStart'].fillna(0) 
    
    # If i_mileage NaN, then use current odometer mileage reading (odomiles)
    merged_df['i_mileage'].fillna(merged_df['odomiles'], inplace=True)

    # Fill NaN values in 'i_months_in_service' with the values from 'daysSinceWarrantyStart' turned into months
    # assuming that the average month is 30.44 days
    # If 'daysSinceWarrantyStart' for a given row is '0', it will set 'i_months_in_service' to '0'.
    # Otherwise, it will set 'i_months_in_service' to '(row['daysSinceWarrantyStart'] / 30.44).round()'
    # if 'i_months_in_service' is NaN. If 'i_months_in_service' is not NaN, it leaves the value as is
    merged_df['i_months_in_service'] = merged_df.apply(
        lambda row: 0 if row['daysSinceWarrantyStart'] == 0 else ((row['daysSinceWarrantyStart'] / 30.44) if pd.isnull(row['i_months_in_service']) and pd.notnull(row['daysSinceWarrantyStart']) else row['i_months_in_service']),
        axis=1
    )

    merged_df['i_months_in_service'] = merged_df['i_months_in_service'].apply(lambda x: round(x) if pd.notnull(x) else x)

    # Fill NaN values in 'i_time_in_service' with the values from 'i_months_in_service' + 1
    # as this seems to be the pattern in the data
    merged_df['i_time_in_service'].fillna(merged_df['i_months_in_service'] + 1, inplace=True)

    na_counts = merged_df.isna().sum()
    na_columns = na_counts[na_counts > 0]

    if len(na_columns) > 0:
        print(f"Data fields with NaN values:\n{na_columns}")
        print(f'Total number of records in the DataFrame: {len(merged_df)}')
    else:
        print('There are no missing values in the DataFrame.')
    
    return merged_df

## Data Normalisation: Standardise Numerical Data

In this step, we are standardising the values of the 'elapsedTimeSec', 'timeSinceLastActivitySec', 'odomiles', 'vehicleAgeAtSession', and 'daysSinceWarrantyStart' columns. These columns represent continuous numerical data (temporal data and odometer readings), which we expect to follow a normal-like distribution.

We are using sklearn's StandardScaler for this task. This method standardizes features by removing the mean and scaling to unit variance. This transformation helps to achieve properties of a standard normal distribution where the mean (average) of each feature is 0 and the standard deviation is 1.

By doing this, we are ensuring that these features have the same scale and thus contributing equally to the model's performance.

In [ ]:
def standardise_num_data(merged_df):
    float_cols = ['elapsedTimeSec', 'timeSinceLastActivitySec', 'odomiles', 'vehicleAgeAtSession',
              'daysSinceWarrantyStart', 'i_mileage', 'i_time_in_service', 'i_months_in_service']
    for col in float_cols:
        merged_df[col] = merged_df[col].astype('float64')
    
    data_scaler = StandardScaler()
    merged_df[float_cols] = data_scaler.fit_transform(merged_df[float_cols])
    return merged_df

## Preprocess and Save Prepared Data

Here we are running all the preprocessing steps on the data using parallel computing capabilities and saving the preprocessed data into a CSV file in the as 'prepared_data.csv' in 'data_out' directory.

In [ ]:
def process_data(filename):
    diagnostic_df = load_data(filename)
    print(f'{filename} - Data import complete\n')
    
    diagnostic_df = initiate_diagnostic_consultation(diagnostic_df)
    print(f'{filename} - Diagnostic consultation initiated\n')
    
    diagnostic_df = derive_vehicle_state_data(diagnostic_df)
    print(f'{filename} - Vehicle state data derived\n')
    
    warranty_df = pd.read_csv(f'data/claims_all.csv', low_memory=False)
    merged_df = merge_diag_warr_data(diagnostic_df, warranty_df)
    print(f'{filename} - Diagnostic and warranty data merged\n')
    
    merged_df = derive_temporal_data_features(merged_df)
    print(f'{filename} - Temporal features derived\n')
    
    merged_df = handle_missing_vals(merged_df)
    print(f'{filename} - Missing values addressed\n')
    
    merged_df = remove_duplicates(merged_df)
    print(f'{filename} - Duplicates removed\n')
    
    merged_df = standardise_num_data(merged_df)
    print(f'{filename} - Numerical data normalised\n')
    
    return merged_df